<a href="https://colab.research.google.com/github/Kushankp/3-Innovative-Machine-Deep-Learning-based-ideas-which-have-an-excellent-real-time-use-case./blob/main/CS483_Colab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS483 - Colab 1
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=09356604194c34aff0e95a2ed1545bb03af3107bc01b6220ebf36f68e9d08486
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter.

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words.

Your outputs will be graded on a range -- if your differences from the ground-truths are within an error threshold of 5, you'll be considered correct.

In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import re

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [6]:
# Loading the input file
input_file = "/content/pg100.txt"
text = sc.textFile(input_file)

In [8]:
# Function to extract words from each line, ensuring lower case and ignoring non-alphabetic words
def extract_words(line):
    # Convert to lower case and use regular expressions to extract words
    words = re.findall(r'\b[a-zA-Z]+\b', line.lower())
    return words

In [9]:
# Transform the input file into a list of words
words_rdd = text.flatMap(extract_words)

In [10]:
# Map each word to a tuple of (first_letter, 1)
letter_count_rdd = words_rdd.map(lambda word: (word[0], 1))

In [11]:
# Reduce by key (letter) to count occurrences
letter_counts = letter_count_rdd.reduceByKey(lambda a, b: a + b)

In [12]:
# Sort the result by letter (key)
sorted_letter_counts = letter_counts.sortByKey()

In [13]:
# Collect and print the results
for letter, count in sorted_letter_counts.collect():
    print(f"{letter}: {count}")

a: 85968
b: 46001
c: 34983
d: 39173
e: 20408
f: 37186
g: 21167
h: 61028
i: 62420
j: 3372
k: 9534
l: 32389
m: 56252
n: 27313
o: 43712
p: 28059
q: 2388
r: 15234
s: 75192
t: 127781
u: 9230
v: 5801
w: 60097
x: 14
y: 25926
z: 79


Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!